# Step 1 - Scraping

- Collect the latest News Title and Paragraph Text

In [ ]:
# Dependencies and setup
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
import requests

In [ ]:
# Set up splinter
executable_path = {"executable_path":ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless = False)

In [ ]:
# URL of page to be scraped
url = "https://redplanetscience.com"
browser.visit(url)

In [ ]:
# Loop through homepage
for x in range(1):
    
    # HTML object
    html = browser.html
    
    # Parse HTML with BeautifulSoup
    soup = bs(html, "html.parser")
    
    # Retrieve all elements that contain News Titles and Paragraph Text
    articles = soup.find_all("div", class_ = "list_text")
    
    # Iterate through each headline
    for article in articles:
        
        # Use BeautifulSoup's find() method to navigate and retrieve attributes
        newsTitle = article.find("div", class_ = "content_title").text
        paragraphText = article.find("div", class_ = "article_teaser_body").text
        print("------------------------------------------")
        print(f"Article:  {newsTitle}")
        print(f"Content:  {paragraphText}")
    
    # Dictionary to be inserted as a MongoDB document
        post = {
            "Title": newsTitle,
            "Paragraph Content": paragraphText,
        }
            
        # collection.insert_one(post)